# Correlating spike trains

Recall that our research questions for this data set were:
- Does stimulation with a sine wave grating increase functional connectivity between neurons?
- If so, are increases in functional connectivity specific to the orientation of the stimulus?

We hypothesized that the answer to both questions would be "yes". Here we compute functional connectivity (correlation) between each pair of neurons in the data set, to answer these questions.

In functional connectivity, we are typically interested in how similar (correlated) the patterns of activity of neurons are over time. That is, if two neurons show increases in spike probabilities at similar times, relative to the presentation of the stimulus, they are said to be more strongly functionally connected than two neurons that do not show similar response patterns. Because the PSTH characterizes the response of a neuron over time, averaged across many trials, these are what we use as input to the functional connectivity analysis. 

### Import packages

We previously talked about  SciPy as the library that provided a function for importing Matlab files. The SciPy library has many useful routines for general-purpose scientific work, including many statistical and machine learning functions. Here we import `scipy.stats` so that we can use its correlation functions. 

~~~python
import scipy.io
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sc
~~~

### Read Data

~~~python
psth = pd.read_csv('data/array_psth.csv', index_col=['orientation', 'channel'])
~~~

## Compute correlation matrix 

pandas has a `.corr()` method that computes a correlation matrix from a DataFrame. It correlates the *columns*, which is not what we want here, because in `psth`, each *row* is a histogram for a channel. So we chain the `.T` method with `.corr()`, which [**transposes**](https://en.wikipedia.org/wiki/Transpose) the matrix (i.e., turns rows into columns).

Below shows the result of doing this, selecting only for the 90 deg orientation condition using `.loc[]`

~~~python
psth.loc[90].T.corr()
~~~

Correlation matrices with large numbers of values are much easier to make sense of when plotted with colour representing the correlation values, rather than as a matrix of raw numbers. Below we do this, separately for each orientation. The plot on the right, below, is the same correlation matrix as shown above, just in a visual rather than tabular format.

~~~python
# Get levels of orientation from the data. Although we know these are 0 and 90, 
# and could hard-code these, our code is more robust and adaptable if we get
# these from the data itself
ortn_levs = psth.index.get_level_values('orientation').unique()

fig = plt.figure(figsize=[20, 7])
for oidx, ortn in enumerate(ortn_levs):
    ax = fig.add_subplot(1, len(ortn_levs), oidx+1)
    plt.imshow(psth.loc[ortn].T.corr(), clim=(-1, 1), cmap='viridis') 
    plt.colorbar()
    ax.invert_yaxis()
    plt.title(str(int(ortn)) + ' deg')
    plt.xlabel('Channel')
    plt.ylabel('Channel')
    
plt.show()
~~~


### Remove noisy channels

The authors who provided the data note that some of the channels were noisy. This may be due to technical issues in recording, or other issues. Regardless, We want to exclude these as their data do not appear to represent real neuronal activity. We first define a list of the noisy channels (provided by Nylen and Wallisch):

~~~python
noisy = [7, 16, 32, 37, 39, 41, 43, 45, 47, 48, 51, 52, 54, 94, 95]
~~~

Since the channels are stored in one of the indexes of the `psth` DataFrame, we use the code below to select and remove the noisy channels. This is similar to code you've seen before, for selecting rows of a pandas DataFrame, using the `~` operator to take the inverse of whatever we select (i.e., "keep the channels that are *not* in `noisy`). 

~~~python
psth = psth[~psth.index.get_level_values('channel').isin(noisy)]
~~~

Now we re-compute and plot the correlation matrices. These look very similar to the ones above, but if you look at the *x* and *y* axis ranges you can see there are fewer channels.

~~~python
# Get levels of orientation from the data. Although we know these are 0 and 90, 
# and could hard-code these, our code is more robust and adaptable if we get
# these from the data itself
ortn_levs = psth.index.get_level_values('orientation').unique()

fig = plt.figure(figsize=[20, 7])
for oidx, ortn in enumerate(ortn_levs):
    ax = fig.add_subplot(1, len(ortn_levs), oidx+1)
    plt.imshow(psth.loc[ortn].T.corr(), clim=(-1, 1), cmap='viridis') 
    plt.colorbar()
    ax.invert_yaxis()
    plt.title(str(int(ortn)) + ' deg')
    plt.xlabel('Channel')
    plt.ylabel('Channel')
    
plt.show()
~~~

### Restrict time range

Finally, recall that the research question refers specifically to functional connectivity when stimuli are presented. So we should restrict the data we use to compute our correlation matrices, to the time points when the stimuli were presented. As noted when we introduced this experiment, each trial was 2150 ms long, but the first 150 ms was a fixation period, followed by a 2000 ms stimulus presentation. So we want to remove the first 150 ms of data.

Since our data are already in the form of histograms, that were computed over bins (ranges of time), we need to know how wide (i.e., how many milliseconds long) each bin is, so that we can figure out how many bins comprise the first 150 ms of the trials.

We know that the total length of the trials was 2150 ms, and we can use the number of columns in the `psth` DataFrame to derive the value for the total number of bins in each PSTH:

~~~python
2150 / psth.shape[1]
~~~

Since this is not an even number, we can round it and save it as the variable `bin_width`:

~~~python
bin_width = round(2150 / psth.shape[1])
print(bin_width)
~~~

We can now use `bin_width` to determine how many bins comprise the 150 ms baseline period:
~~~python
baseline = round(150 / bin_width)
print(baseline)
~~~

So now we can use `.iloc[]` to select only the column numbers (bins) after the baseline:
~~~python
psth_stim = psth.iloc[:, baseline:]
psth_stim
~~~

~~~python
psth_stim = psth.iloc[:, baseline:]

fig = plt.figure(figsize=[20, 7])
for oidx, ortn in enumerate(ortn_levs):
    ax = fig.add_subplot(1, len(ortn_levs), oidx+1)
    plt.imshow(psth_stim.loc[ortn].T.corr(), clim=(-1, 1), cmap='viridis') 
    plt.colorbar()
    ax.invert_yaxis()
    plt.title(str(int(ortn)) + ' deg')
    plt.xlabel('Channel')
    plt.ylabel('Channel')
    
plt.show()
~~~